# Age model with IMDB & UTKface

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install git+https://github.com/qubvel/segmentation_models

  Cloning https://github.com/qubvel/segmentation_models to /tmp/pip-req-build-ltavluq3
  Running command git clone -q https://github.com/qubvel/segmentation_models /tmp/pip-req-build-ltavluq3
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 51kB 5.7MB/s 
  Created wheel for segmentation-models: filename=segmentation_models-1.0.1-cp37-none-any.whl size=33794 sha256=8f0a60c629bbc18dab218e5d51df099726c6b5dddd6221a4ed1ff56dd95f9b22
  Stored in directory: /tmp/pip-ephem-wheel-cache-sds07l4z/wheels/49/cf/46/cbb4bb64518c402aea99df9d466f1081450597e653256bbcf4
Successfully built segmentation-models


## config

In [10]:
NUM_SAMPLE = 45000
NUM_AGE_CLASSES = 3
NUM_GENDER_CLASSES = 2
IMAGE_SIZE = 56
BATCH_SIZE = 256
WEIGHT_INIT = 0.08
DROPOUT_RATE = 0.2
input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)
NUM_EPOCHS = 40
DECAY_LR_RATE = 0.9
MODEL_PATH = '/content/drive/MyDrive/3033proj/weights/weight_age_estimation_imbalanced_epoch50.h5'

CLASS_SAMPLE = 15000

In [16]:
model_name = 'b4'
dir = '/content/drive/MyDrive/3033proj/model_log/'
log_path = dir+model_name+'_log.csv'
log_path

'/content/drive/MyDrive/3033proj/model_log/b4_log.csv'

## data

In [3]:
import keras
# import config as cf
import os
import numpy as np

class Datasets(object):
    def __init__(self,data_name):
        self.data_name=data_name
        self.datasets = self.getData(self)
        self.final_data = []
        self.convert_data_format()


    def gen(self):
        # np.random.shuffle(self.final_data)

        images = []
        age_labels = []
        gender_labels = []

        for i in range(len(self.final_data)):
            image, age, gender = self.final_data[i]
            images.append(image)
            age_labels.append(age)
            gender_labels.append(gender)

        age_labels = keras.utils.to_categorical(age_labels, num_classes=NUM_AGE_CLASSES)
        gender_labels = keras.utils.to_categorical(gender_labels, num_classes=NUM_GENDER_CLASSES)
        return images, age_labels, gender_labels

    @staticmethod
    def getData(self):
        print('Loading age image...')
        # data_3
        if self.data_name=='data_3':
          data = np.load(os.path.join(os.getcwd(), '/content/drive/MyDrive/3033proj/data_3.npy'), allow_pickle=True) 
        if self.data_name=='data_5':
          data = np.load(os.path.join(os.getcwd(), '/content/drive/MyDrive/3033proj/data_5.npy'), allow_pickle=True) 
        if self.data_name=='data_10':
          data = np.load(os.path.join(os.getcwd(), '/content/drive/MyDrive/3033proj/data_10.npy'), allow_pickle=True)
        np.random.shuffle(data)
        all_data = []
        # random select 45000 data
        for i in range(45000):  ### number of samples
            all_data.append(data[i])

        print('Number of age data:', str(len(all_data)))

        return all_data

    def convert_data_format(self):
        # Age datasets:
        for i in range(len(self.datasets)):
            image = self.datasets[i][0] / 255.0
            age_labels = self.datasets[i][1]
            gender_labels = self.datasets[i][2]
            self.final_data.append((image, age_labels, gender_labels))

## model

In [4]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras import optimizers
from keras.utils.generic_utils import get_custom_objects
from keras.layers import GlobalMaxPool2D, Dropout, Dense, Activation, BatchNormalization
from keras.models import Model
import numpy as np
import pandas as pd
# import config as cf
from keras.backend import sigmoid
# Options: EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5,...
# Higher the number, the more complex the model is
# Now in this project, I use model EfficientNetB4
from efficientnet.keras import EfficientNetB4 as NetB4
from efficientnet.keras import EfficientNetB4 as NetB5
from efficientnet.keras import EfficientNetB4 as NetB6
from efficientnet.keras import EfficientNetB4 as NetB7

In [11]:
class SwishActivation(Activation):
    def __init__(self, activation, **kwargs):
        super(SwishActivation, self).__init__(activation, **kwargs)
        self.__name__ = 'swish_act'

def swish_act(x, beta=1):
    return x * sigmoid(beta * x)

class Efficient_Net(object):
    def __init__(self, model_name, data_name, trainable=True):
        self.trainable = trainable
        self.model_name=model_name
        self.data_name=data_name
        if self.trainable: 
            self.train_data = Datasets(self.data_name)
        self.model = self.build_model(self.model_name)

        # Compile the model
        # losses = {
        #     "age_output": "categorical_crossentropy",
        #     "gender_output": "categorical_crossentropy"
        # }

        opt = optimizers.Adam(1e-3)
        self.model.compile(loss=keras.losses.categorical_crossentropy, optimizer=opt, metrics=['acc'])

        # Train the part you added
        if self.trainable:
            self.model.summary()

    @staticmethod
    def build_age_branch(x):
        # Output age branch
        predictions_age = Dense(NUM_AGE_CLASSES, activation="softmax", name='age_output')(x)

        return predictions_age

    @staticmethod
    def build_gender_branch(x):
        # Output gender branch
        predictions_gender = Dense(NUM_GENDER_CLASSES, activation="softmax", name='gender_output')(x)

        return predictions_gender

    def build_model(self,model_name):

        get_custom_objects().update({'swish_act': SwishActivation(swish_act)})

        # Model
        if model_name=='B4':
          model = NetB4(weights='imagenet', include_top=False, input_shape=input_shape)
        if model_name=='B5':
          model = NetB5(weights='imagenet', include_top=False, input_shape=input_shape)
        if model_name=='B6':
          model = NetB6(weights='imagenet', include_top=False, input_shape=input_shape)
        if model_name=='B7':
          model = NetB7(weights='imagenet', include_top=False, input_shape=input_shape)
        

        # Adding 1 fully-connected layers to B4.
        x = model.output

        x = BatchNormalization()(x)
        x = GlobalMaxPool2D(name='gap1')(x) 
        x = Dropout(DROPOUT_RATE, name='dropout1')(x)

        # Output layer
        predictions_age = self.build_age_branch(x)
        # predictions_gender = self.build_gender_branch(x)
        model_final = Model(inputs=model.input, outputs=predictions_age)

        return model_final

    def train(self):
        # reduce learning rate
        reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=DECAY_LR_RATE, patience=5, verbose=1, )
        # Model Checkpoint
        # cpt_save = ModelCheckpoint('/content/drive/MyDrive/3033proj/weights/weight_a_g3_B5.h5', save_best_only=True, monitor='val_age_output_acc', mode='max')
        print("Training......")

        trainX, trainAgeY, _ = self.train_data.gen()
        trainX = np.array(trainX)

        history = self.model.fit(trainX, trainAgeY, validation_split=0.2,
                       callbacks=reduce_lr, verbose=1, epochs=NUM_EPOCHS, shuffle=True,
                       batch_size=BATCH_SIZE)
        
        dir = '/content/drive/MyDrive/3033proj/model_log/'
        log_path = dir+self.model_name+self.data_name+'_log.csv'
        hist = pd.DataFrame(history.history)
        hist.to_csv(log_path)
        

# IMDB-WIKI

## B4+data_3

In [6]:
model = Efficient_Net('B4', 'data_3', trainable=True)

Loading age image...
Number of age data: 45000
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
_______________________________________________

In [7]:
model.train()

Training......
Epoch 1/40
141/141 [==============================] - 67s 304ms/step - loss: 1.4696 - acc: 0.5433 - val_loss: 1.8226 - val_acc: 0.6378
Epoch 2/40
141/141 [==============================] - 38s 270ms/step - loss: 0.8772 - acc: 0.6638 - val_loss: 3.2382 - val_acc: 0.6597
Epoch 3/40
141/141 [==============================] - 38s 270ms/step - loss: 0.8387 - acc: 0.6622 - val_loss: 0.8113 - val_acc: 0.6633
Epoch 4/40
141/141 [==============================] - 38s 270ms/step - loss: 0.8643 - acc: 0.6616 - val_loss: 0.8580 - val_acc: 0.6604
Epoch 5/40
141/141 [==============================] - 38s 270ms/step - loss: 0.9001 - acc: 0.6607 - val_loss: 3.2253 - val_acc: 0.6613
Epoch 6/40
141/141 [==============================] - 38s 270ms/step - loss: 0.8559 - acc: 0.6681 - val_loss: 0.8252 - val_acc: 0.6616
Epoch 7/40
141/141 [==============================] - 38s 270ms/step - loss: 0.8573 - acc: 0.6652 - val_loss: 0.9541 - val_acc: 0.6619
Epoch 8/40
141/141 [====================

## B5+data_3

In [12]:
NUM_AGE_CLASSES = 3
model = Efficient_Net('B5', 'data_3', trainable=True)
model.train()

Loading age image...
Number of age data: 45000
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_3[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
_____________________________________________

## B6+data_3

In [13]:
NUM_AGE_CLASSES = 3
model = Efficient_Net('B6', 'data_3', trainable=True)
model.train()

Loading age image...
Number of age data: 45000
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_4[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
_____________________________________________

## B7+data_3

In [14]:
NUM_AGE_CLASSES = 3
model = Efficient_Net('B7', 'data_3', trainable=True)
model.train()

Loading age image...
Number of age data: 45000
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_5[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
_____________________________________________

## B4+data_5

In [15]:
NUM_AGE_CLASSES = 5
model = Efficient_Net('B4', 'data_5', trainable=True)
model.train()

Loading age image...
Number of age data: 45000
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_6[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
_____________________________________________

## B5+data_5

In [16]:
NUM_AGE_CLASSES = 5
model = Efficient_Net('B5', 'data_5', trainable=True)
model.train()

Loading age image...
Number of age data: 45000
Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_7[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
_____________________________________________

## B6+data_5

In [17]:
NUM_AGE_CLASSES = 5
model = Efficient_Net('B6', 'data_5', trainable=True)
model.train()

Loading age image...
Number of age data: 45000
Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_8[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
_____________________________________________

## B7+data_5

In [18]:
NUM_AGE_CLASSES = 5
model = Efficient_Net('B7', 'data_5', trainable=True)
model.train()

Loading age image...
Number of age data: 45000
Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_9[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
_____________________________________________

## B4+data_10

In [8]:
NUM_SAMPLE = 45000
NUM_AGE_CLASSES = 10
NUM_GENDER_CLASSES = 2
IMAGE_SIZE = 56
BATCH_SIZE = 256
WEIGHT_INIT = 0.08
DROPOUT_RATE = 0.2
input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)
NUM_EPOCHS = 40
DECAY_LR_RATE = 0.9
# MODEL_PATH = '/content/drive/MyDrive/3033proj/weights/weight_age_estimation_imbalanced_epoch50.h5'

CLASS_SAMPLE = 15000

In [9]:
model = Efficient_Net('B4', 'data_10', trainable=True)

Loading age image...
Number of age data: 45000
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
_____________________________________________

In [10]:
model.train()

Training......
Epoch 1/40
141/141 [==============================] - 60s 290ms/step - loss: 2.7777 - acc: 0.2098 - val_loss: 5.0294 - val_acc: 0.1788
Epoch 2/40
141/141 [==============================] - 38s 270ms/step - loss: 1.9823 - acc: 0.2601 - val_loss: 1.8711 - val_acc: 0.2593
Epoch 3/40
141/141 [==============================] - 38s 270ms/step - loss: 1.9597 - acc: 0.2600 - val_loss: 1.9630 - val_acc: 0.2643
Epoch 4/40
141/141 [==============================] - 38s 270ms/step - loss: 1.9744 - acc: 0.2571 - val_loss: 1.9077 - val_acc: 0.2548
Epoch 5/40
141/141 [==============================] - 38s 270ms/step - loss: 1.9891 - acc: 0.2539 - val_loss: 2.3902 - val_acc: 0.2211
Epoch 6/40
141/141 [==============================] - 38s 270ms/step - loss: 2.0401 - acc: 0.2499 - val_loss: 1.8810 - val_acc: 0.2560
Epoch 7/40
141/141 [==============================] - 38s 270ms/step - loss: 1.9855 - acc: 0.2532 - val_loss: 1.8669 - val_acc: 0.2553
Epoch 8/40
141/141 [====================

## B5+data_10

In [19]:
NUM_AGE_CLASSES = 10
model = Efficient_Net('B5', 'data_10', trainable=True)
model.train()  

Loading age image...
Number of age data: 45000
Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_10[0][0]                   
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
_____________________________________________

## B6+data_10

In [20]:
NUM_AGE_CLASSES = 10
model = Efficient_Net('B6', 'data_10', trainable=True)
model.train()  

Loading age image...
Number of age data: 45000
Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_11[0][0]                   
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
____________________________________________

## B7+data10

In [21]:
NUM_AGE_CLASSES = 10
model = Efficient_Net('B7', 'data_10', trainable=True)
model.train()  

Loading age image...
Number of age data: 45000
Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_12[0][0]                   
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
____________________________________________

# UTKface dataset

In [6]:
import keras
# import config as cf
import os
import numpy as np

class Datasets(object):
    def __init__(self,data_name):
        self.data_name=data_name
        self.datasets = self.getData(self)
        self.final_data = []
        self.convert_data_format()


    def gen(self):
        # np.random.shuffle(self.final_data)

        images = []
        age_labels = []
        gender_labels = []

        for i in range(len(self.final_data)):
            image, age, gender = self.final_data[i]
            images.append(image)
            age_labels.append(age)
            gender_labels.append(gender)

        age_labels = keras.utils.to_categorical(age_labels, num_classes=NUM_AGE_CLASSES)
        gender_labels = keras.utils.to_categorical(gender_labels, num_classes=NUM_GENDER_CLASSES)
        return images, age_labels, gender_labels

    @staticmethod
    def getData(self):
        print('Loading age image...')
        # data_3
        if self.data_name=='data_3':
          data = np.load(os.path.join(os.getcwd(), '/content/drive/MyDrive/3033proj/data_3_utk.npy'), allow_pickle=True) 
        if self.data_name=='data_5':
          data = np.load(os.path.join(os.getcwd(), '/content/drive/MyDrive/3033proj/data_5_utk.npy'), allow_pickle=True) 
        if self.data_name=='data_10':
          data = np.load(os.path.join(os.getcwd(), '/content/drive/MyDrive/3033proj/data_10_utk.npy'), allow_pickle=True)
        np.random.shuffle(data)
        all_data = []
        # random select 23165 data
        for i in range(23165):  ### number of samples
            all_data.append(data[i])

        print('Number of age data:', str(len(all_data)))

        return all_data

    def convert_data_format(self):
        # Age datasets:
        for i in range(len(self.datasets)):
            image = self.datasets[i][0] / 255.0
            age_labels = self.datasets[i][1]
            gender_labels = self.datasets[i][2]
            self.final_data.append((image, age_labels, gender_labels))

In [7]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras import optimizers
from keras.utils.generic_utils import get_custom_objects
from keras.layers import GlobalMaxPool2D, Dropout, Dense, Activation, BatchNormalization
from keras.models import Model
import numpy as np
import pandas as pd
# import config as cf
from keras.backend import sigmoid
# Options: EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5,...
# Higher the number, the more complex the model is
# Now in this project, I use model EfficientNetB4
from efficientnet.keras import EfficientNetB4 as NetB4
from efficientnet.keras import EfficientNetB4 as NetB5
from efficientnet.keras import EfficientNetB4 as NetB6
from efficientnet.keras import EfficientNetB4 as NetB7

In [12]:
class SwishActivation(Activation):
    def __init__(self, activation, **kwargs):
        super(SwishActivation, self).__init__(activation, **kwargs)
        self.__name__ = 'swish_act'

def swish_act(x, beta=1):
    return x * sigmoid(beta * x)

class Efficient_Net(object):
    def __init__(self, model_name, data_name, trainable=True):
        self.trainable = trainable
        self.model_name=model_name
        self.data_name=data_name
        if self.trainable: 
            self.train_data = Datasets(self.data_name)
        self.model = self.build_model(self.model_name)

        # Compile the model
        # losses = {
        #     "age_output": "categorical_crossentropy",
        #     "gender_output": "categorical_crossentropy"
        # }

        opt = optimizers.Adam(1e-3)
        self.model.compile(loss=keras.losses.categorical_crossentropy, optimizer=opt, metrics=['acc'])

        # Train the part you added
        if self.trainable:
            self.model.summary()

    @staticmethod
    def build_age_branch(x):
        # Output age branch
        predictions_age = Dense(NUM_AGE_CLASSES, activation="softmax", name='age_output')(x)

        return predictions_age

    @staticmethod
    def build_gender_branch(x):
        # Output gender branch
        predictions_gender = Dense(NUM_GENDER_CLASSES, activation="softmax", name='gender_output')(x)

        return predictions_gender

    def build_model(self,model_name):

        get_custom_objects().update({'swish_act': SwishActivation(swish_act)})

        # Model
        if model_name=='B4':
          model = NetB4(weights='imagenet', include_top=False, input_shape=input_shape)
        if model_name=='B5':
          model = NetB5(weights='imagenet', include_top=False, input_shape=input_shape)
        if model_name=='B6':
          model = NetB6(weights='imagenet', include_top=False, input_shape=input_shape)
        if model_name=='B7':
          model = NetB7(weights='imagenet', include_top=False, input_shape=input_shape)
        

        # Adding 1 fully-connected layers to B4.
        x = model.output

        x = BatchNormalization()(x)
        x = GlobalMaxPool2D(name='gap1')(x) 
        x = Dropout(DROPOUT_RATE, name='dropout1')(x)

        # Output layer
        predictions_age = self.build_age_branch(x)
        # predictions_gender = self.build_gender_branch(x)
        model_final = Model(inputs=model.input, outputs=predictions_age)

        return model_final

    def train(self):
        # reduce learning rate
        reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=DECAY_LR_RATE, patience=5, verbose=1, )
        # Model Checkpoint
        cpt_save = ModelCheckpoint('/content/drive/MyDrive/3033proj/weights/weight_age_data3_B4.h5', save_best_only=True, monitor='val_acc', mode='max')
        print("Training......")

        trainX, trainAgeY, _ = self.train_data.gen()
        trainX = np.array(trainX)

        history = self.model.fit(trainX, trainAgeY, validation_split=0.2,
                       callbacks=[cpt_save,reduce_lr], verbose=1, epochs=NUM_EPOCHS, shuffle=True,
                       batch_size=BATCH_SIZE)
        
        dir = '/content/drive/MyDrive/3033proj/model_log/'
        log_path = dir+self.model_name+self.data_name+'utk_log.csv'
        hist = pd.DataFrame(history.history)
        # hist.to_csv(log_path)
        

## B4+data_3 

In [9]:
NUM_AGE_CLASSES = 3
model = Efficient_Net('B4', 'data_3', trainable=True)
model.train()  

Loading age image...
Number of age data: 23165
71892992/71892840 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]   

## B5+data_3

In [10]:
NUM_AGE_CLASSES = 3
model = Efficient_Net('B5', 'data_3', trainable=True)
model.train()  

Loading age image...
Number of age data: 23165
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
_____________________________________________

## B6+data_3

In [11]:
NUM_AGE_CLASSES = 3
model = Efficient_Net('B6', 'data_3', trainable=True)
model.train()  

Loading age image...
Number of age data: 23165
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_3[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
_____________________________________________

## B7+data_3

In [12]:
NUM_AGE_CLASSES = 3
model = Efficient_Net('B7', 'data_3', trainable=True)
model.train()  

Loading age image...
Number of age data: 23165
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_4[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
_____________________________________________

## B4+data_5

In [16]:
NUM_AGE_CLASSES = 5
model = Efficient_Net('B4', 'data_5', trainable=True)
model.train()  

Loading age image...
Number of age data: 23165
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_6[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
_____________________________________________

## B5+data_5

In [18]:
NUM_AGE_CLASSES = 5
model = Efficient_Net('B5', 'data_5', trainable=True)
model.train()  

Loading age image...
Number of age data: 23165
Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_8[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
_____________________________________________

## B6+data_5

In [19]:
NUM_AGE_CLASSES = 5
model = Efficient_Net('B6', 'data_5', trainable=True)
model.train()  

Loading age image...
Number of age data: 23165
Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_9[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
_____________________________________________

## B7+data_5

In [20]:
NUM_AGE_CLASSES = 5
model = Efficient_Net('B7', 'data_5', trainable=True)
model.train()  

Loading age image...
Number of age data: 23165
Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_10[0][0]                   
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
_____________________________________________

## B4+data_10

In [21]:
NUM_AGE_CLASSES = 10
model = Efficient_Net('B4', 'data_10', trainable=True)
model.train()  

Loading age image...
Number of age data: 23165
Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_11[0][0]                   
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
____________________________________________

## B5+data_10

In [22]:
NUM_AGE_CLASSES = 10
model = Efficient_Net('B5', 'data_10', trainable=True)
model.train()  

Loading age image...
Number of age data: 23165
Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_12[0][0]                   
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
____________________________________________

## B6+data_10

In [23]:
NUM_AGE_CLASSES = 10
model = Efficient_Net('B6', 'data_10', trainable=True)
model.train()  

Loading age image...
Number of age data: 23165
Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_13[0][0]                   
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
____________________________________________

## B7+data_10

In [24]:
NUM_AGE_CLASSES = 10
model = Efficient_Net('B7', 'data_10', trainable=True)
model.train()  

Loading age image...
Number of age data: 23165
Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 28, 28, 48)   1296        input_14[0][0]                   
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 28, 28, 48)   192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 28, 28, 48)   0           stem_bn[0][0]                    
____________________________________________